# <u><b>Neural Style Transfer:</b></u>
<h3> -------- Author- Amar Choudhary ----------- </h3>

## Introduction
In this project , we make or transform a new image which has features of both content and style image from both content and style image that we pass in model. 

For this we have to do fine tunning on vgg19 model and calculate both content loss and style loss.

![Flow Diagram](https://www.researchgate.net/publication/347180236/figure/fig3/AS:1022415605886976@1620774284899/Model-Optimisation-Based-Neural-Style-Transfer-System-5.jpg)


## *So we got Total Loss :*

*     Total Loss = Content Loss + Style Loss


## The process involves three key steps:

* *Content Extraction:* A content image is processed through the CNN to capture its structure and essential details.

* *Style Extraction:* A style image is passed through the same network to extract style features, including textures and colors, using Gram matrices from different layers.

* *Optimization:* Starting with a copy of the content image, an iterative optimization process adjusts this image to minimize the loss function. This loss combines content loss (difference from the original content image) and style loss (difference from the style image).

## <i>We got final Results as :</i>


![3](../markdown/3.png)

![7](https://github.com/AmarBackInField/NeuralStyleTransfer-v1.0/assets/126746349/3803131e-996f-4f5f-b3b1-f4d2c213a76a)

![9](https://github.com/AmarBackInField/NeuralStyleTransfer-v1.0/assets/126746349/9dc3fc70-7a7b-466f-be36-fdbc101b19c7)







# Importing Libraries

In [7]:
# -- author --- Amar Choudhary

import tensorflow as tf
import keras.preprocessing.image as process_im
from PIL import Image
import matplotlib.pyplot as plt
from keras.applications import VGG19
import numpy as np
from keras.models import Model
from tensorflow.python.keras import models
from tensorflow.python.keras import losses
from tensorflow.python.keras import layers
from tensorflow.python.keras import backend as K
import functools
import IPython.display
import os
import cv2
import random

# Downloading Dataset from KAGGLE

In [2]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [3]:
!kaggle datasets download -d ikarus777/best-artworks-of-all-time

Dataset URL: https://www.kaggle.com/datasets/ikarus777/best-artworks-of-all-time
License(s): CC-BY-NC-SA-4.0
100% 2.29G/2.29G [00:31<00:00, 133MB/s]
100% 2.29G/2.29G [00:31<00:00, 77.0MB/s]


In [8]:
!kaggle datasets download -d duttadebadri/image-classification

Dataset URL: https://www.kaggle.com/datasets/duttadebadri/image-classification
License(s): CC0-1.0
image-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


# Unzipping All Datasets

In [ ]:
# Unzipping All Data (content + Style)
!unzip best-artworks-of-all-time.zip
!unzip image-classification.zip

In [10]:
# extracting artists names
list_of_artists=os.listdir("images/images")

In [11]:
print(len(list_of_artists)) # NUMBER OF ARTISTS
print(list_of_artists)

55
['Gustav_Klimt', 'Gustave_Courbet', 'Pierre-Auguste_Renoir', 'Henri_de_Toulouse-Lautrec', 'Peter_Paul_Rubens', 'Caravaggio', 'Francisco_Goya', 'Edouard_Manet', 'Albrecht_DuтХа├кrer', 'Mikhail_Vrubel', 'Vincent_van_Gogh', 'Titian', 'Edgar_Degas', 'food and d rinks', 'Frida_Kahlo', 'Sandro_Botticelli', 'Henri_Rousseau', 'Eugene_Delacroix', 'Michelangelo', 'Raphael', 'Vasiliy_Kandinskiy', 'Albrecht_Du╠Иrer', 'Andrei_Rublev', 'Claude_Monet', 'Giotto_di_Bondone', 'Leonardo_da_Vinci', 'Hieronymus_Bosch', 'Camille_Pissarro', 'Rembrandt', 'Marc_Chagall', 'Andy_Warhol', 'Paul_Cezanne', 'William_Turner', 'Paul_Klee', 'Diego_Velazquez', 'art and culture', 'Jackson_Pollock', 'Joan_Miro', 'Pieter_Bruegel', 'Kazimir_Malevich', 'Henri_Matisse', 'Pablo_Picasso', 'Alfred_Sisley', 'Diego_Rivera', 'Georges_Seurat', 'Piet_Mondrian', 'Amedeo_Modigliani', 'Salvador_Dali', 'Paul_Gauguin', 'El_Greco', 'Jan_van_Eyck', 'architecure', 'Edvard_Munch', 'travel and  adventure', 'Rene_Magritte']


In [12]:
def img_by_artist(path_dir):
  list_of_artists=os.listdir("images/images") # 55 artists list
  fig,axs=plt.subplots(5,5,figsize=(30,30)) # PLOTING IN 5 ROWS AND 5 COLUMNS
  # plt.figure(figsize=(50,50))
  for i in range(5):
    artist=random.choice(list_of_artists) # Picking_up one artist
    path=os.path.join(path_dir,artist)
    path_list=os.listdir(path) # Corresponding to that artist it no. of painting path
    for j in range(5):
      path_of_image=random.choice(path_list)
      full_path_dir=os.path.join("images/images",artist,path_of_image)
      img_arr=cv2.imread(full_path_dir)
      img_arr=cv2.resize(img_arr,(256,256))
      axs[i,j].imshow(img_arr)
      plt.title(artist)
    print('\n')

In [13]:
def show_im(img,title=None):
    img=np.squeeze(img,axis=0) #squeeze array to drop batch axis
    plt.imshow(np.uint8(img))
    if title is None:
        pass
    else:
        plt.title(title)
    plt.imshow(np.uint8(img))

In [ ]:
img_by_artist("images/images")

![output](markdown\1.png)

In [15]:
def load_file(image_path):
    image =  Image.open(image_path)
    max_dim=512
    factor=max_dim/max(image.size)
    image=image.resize((round(image.size[0]*factor),round(image.size[1]*factor)),Image.ANTIALIAS)
    im_array = process_im.img_to_array(image)
    im_array = np.expand_dims(im_array,axis=0)
    return im_array

In [47]:
content_path="validation/validation/art and culture/0002.jpg"
style_path="images/images/Alfred_Sisley/Alfred_Sisley_10.jpg"

In [48]:
content = load_file(content_path)
style = load_file(style_path)

<ipython-input-15-66992c8f8f0a>:5: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image=image.resize((round(image.size[0]*factor),round(image.size[1]*factor)),Image.ANTIALIAS)


In [ ]:
plt.figure(figsize=(10,10))
content = load_file(content_path)
style = load_file(style_path)
plt.subplot(1,2,1)
show_im(content,'Content Image')
plt.subplot(1,2,2)
show_im(style,'Style Image')
plt.show()

# Declaring Functions

In [ ]:
def img_preprocess(img_path):
    image=load_file(img_path) # Converting into image array
    img=tf.keras.applications.vgg19.preprocess_input(image)
    return img

In [ ]:
def deprocess_img(processed_img):
  x = processed_img.copy()
  if len(x.shape) == 4:
    x = np.squeeze(x, 0)
  assert len(x.shape) == 3 #Input dimension must be [1, height, width, channel] or [height, width, channel]


  # perform the inverse of the preprocessing step
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = x[:, :, ::-1] # converting BGR to RGB channel

  x = np.clip(x, 0, 255).astype('uint8')
  return x

![Flow Diagram](https://www.researchgate.net/publication/347180236/figure/fig3/AS:1022415605886976@1620774284899/Model-Optimisation-Based-Neural-Style-Transfer-System-5.jpg)

In [ ]:
content_layers = ['block5_conv2']
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
number_content=len(content_layers)
number_style =len(style_layers)

In [ ]:
def get_model():
    vgg=tf.keras.applications.vgg19.VGG19(include_top=False,weights='imagenet')
    vgg.trainable=False
    content_output=[vgg.get_layer(layer).output for layer in content_layers]
    style_output=[vgg.get_layer(layer).output for layer in style_layers]
    model_output= style_output+content_output
    return models.Model(vgg.input,model_output)

# Layers of VGG19 Model

In [ ]:
model=VGG19(include_top=False,weights='imagenet')
model.summary()

In [ ]:
for layer in model.layers:
  print(layer)

# Model with having Content and Style Layers only
* content_layers =
                ['block5_conv2']
* style_layers =
                ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']


In [ ]:
model=get_model()
model.summary()

# Defining Content and style loss

![Coontent Loss](https://cdn-images-1.medium.com/max/800/1*1YfGhmzBw7EK3e8CRpZbuA.png)

In [ ]:
def get_content_loss(noise,target):
    loss = tf.reduce_mean(tf.square(noise-target))
    return loss

![Gram Matrix](https://cdn-images-1.medium.com/max/800/1*5xx9KmhVb59Mxe_buOwHBA.png)

![Style loss](https://cdn-images-1.medium.com/max/800/1*PuYveCM2BlgFfjUCr6I_Ng.png)

In [58]:
def gram_matrix(tensor):
    channels=int(tensor.shape[-1])
    vector=tf.reshape(tensor,[-1,channels])
    n=tf.shape(vector)[0]
    gram_matrix=tf.matmul(vector,vector,transpose_a=True)
    return gram_matrix/tf.cast(n,tf.float32)

def get_style_loss(noise,target):
    gram_noise=gram_matrix(noise)
    #gram_target=gram_matrix(target)
    loss=tf.reduce_mean(tf.square(target-gram_noise))
    return loss

In [59]:
def get_features(model,content_path,style_path):
    content_img=img_preprocess(content_path)
    style_image=img_preprocess(style_path)

    content_output=model(content_img)
    style_output=model(style_image)

    content_feature = [layer[0] for layer in content_output[number_style:]]
    style_feature = [layer[0] for layer in style_output[:number_style]]
    return content_feature,style_feature

In [60]:
def compute_loss(model, loss_weights,image, gram_style_features, content_features):
    style_weight,content_weight = loss_weights #style weight and content weight are user given parameters
                                               #that define what percentage of content and/or style will be preserved in the generated image

    output=model(image)
    content_loss=0
    style_loss=0

    noise_style_features = output[:number_style]
    noise_content_feature = output[number_style:]

    weight_per_layer = 1.0/float(number_style)
    for a,b in zip(gram_style_features,noise_style_features):
        style_loss+=weight_per_layer*get_style_loss(b[0],a)


    weight_per_layer =1.0/ float(number_content)
    for a,b in zip(noise_content_feature,content_features):
        content_loss+=weight_per_layer*get_content_loss(a[0],b)

    style_loss *= style_weight
    content_loss *= content_weight

    total_loss = content_loss + style_loss


    return total_loss,style_loss,content_loss

In [61]:
def compute_grads(dictionary):
    with tf.GradientTape() as tape:
        all_loss=compute_loss(**dictionary)

    total_loss=all_loss[0]
    return tape.gradient(total_loss,dictionary['image']),all_loss

In [69]:
def run_style_transfer(content_path,style_path,epochs=500,content_weight=1e3, style_weight=1e-2):

    model=get_model()

    for layer in model.layers:
        layer.trainable = False

    content_feature,style_feature = get_features(model,content_path,style_path)
    style_gram_matrix=[gram_matrix(feature) for feature in style_feature]

    noise = img_preprocess(content_path)
    noise=tf.Variable(noise,dtype=tf.float32)

    optimizer = tf.keras.optimizers.Adam(learning_rate=5, beta_1=0.99, epsilon=1e-1)

    best_loss,best_img=float('inf'),None

    loss_weights = (style_weight, content_weight)
    dictionary={'model':model,
              'loss_weights':loss_weights,
              'image':noise,
              'gram_style_features':style_gram_matrix,
              'content_features':content_feature}

    norm_means = np.array([103.939, 116.779, 123.68])
    min_vals = -norm_means
    max_vals = 255 - norm_means

    imgs = []
    for i in range(epochs):
        grad,all_loss=compute_grads(dictionary)
        total_loss,style_loss,content_loss=all_loss
        optimizer.apply_gradients([(grad,noise)])
        clipped=tf.clip_by_value(noise,min_vals,max_vals)
        noise.assign(clipped)

        if total_loss<best_loss:
            best_loss = total_loss
            best_img = deprocess_img(noise.numpy())

         #for visualization

    #     if i%5==0:
    #         plot_img = noise.numpy()
    #         plot_img = deprocess_img(plot_img)
    #         imgs.append(plot_img)
    #         IPython.display.clear_output(wait=True)
    #         IPython.display.display_png(Image.fromarray(plot_img))
    #         print('Epoch: {}'.format(i))
    #         print('Total loss: {:.4e}, '
    #           'style loss: {:.4e}, '
    #           'content loss: {:.4e}, '.format(total_loss, style_loss, content_loss))

    # IPython.display.clear_output(wait=True)


    return best_img,best_loss,imgs

In [70]:
best, best_loss,image = run_style_transfer(content_path,style_path, epochs=500)

<ipython-input-15-66992c8f8f0a>:5: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image=image.resize((round(image.size[0]*factor),round(image.size[1]*factor)),Image.ANTIALIAS)


In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(1,3,3)
plt.imshow(best)
plt.title('Style transfer Image')
plt.xticks([])
plt.yticks([])
plt.subplot(1,3,1)
show_im(content,'Content Image')
plt.xticks([])
plt.yticks([])
plt.subplot(1,3,2)
show_im(style,'Style Image')
plt.xticks([])
plt.yticks([])
plt.show()

![output1](markdown\2.png)

In [65]:
content.shape

(1, 512, 403, 3)

In [72]:
# Here content , style are path in string format
def plot_result(best,content,style):
  plt.figure(figsize=(15,15))
  plt.subplot(1,3,3)
  plt.imshow(best)
  plt.title('Style transfer Image')
  plt.xticks([])
  plt.yticks([])
  plt.subplot(1,3,1)
  show_im(content,'Content Image')
  plt.xticks([])
  plt.yticks([])
  plt.subplot(1,3,2)
  show_im(style,'Style Image')
  plt.xticks([])
  plt.yticks([])
  plt.show()

In [73]:
def find_result():
  artist=random.choice(os.listdir("images/images"))
  path=os.path.join("images/images",artist)
  items=os.listdir(path) # In list format

  category=random.choice(os.listdir("validation/validation"))
  path1=os.path.join("validation/validation",category)
  items1=os.listdir(path1)
  for i in range(5):
    choose_one_item_at_time_in_style=random.choice(items)
    choose_one_item_At_time_in_style_dir=os.path.join(path,choose_one_item_at_time_in_style)
    style_path=choose_one_item_At_time_in_style_dir # style path
    style=load_file(style_path)
    choose_one_item_at_time_in_content=random.choice(items1)
    choose_one_item_At_time_in_content_dir=os.path.join(path1,choose_one_item_at_time_in_content)
    content_path=choose_one_item_At_time_in_content_dir # content path
    content=load_file(content_path)
    best, best_loss,image = run_style_transfer(content_path,style_path, epochs=500)
    plot_result(best,content,style)

In [ ]:
find_result()

![output1](markdown\2.png)
![output1](markdown\3.png)
![output1](markdown\5.png)
![output1](markdown\7.png)
![output1](markdown\8.png)
![output1](markdown\9.png)

# Resource

* For Dataset - https://www.kaggle.com/datasets
* Taking Refrence Official Papers - https://arxiv.org/pdf/1508.06576.pdf https://arxiv.org/pdf/1603.08155.pdf
* https://towardsdatascience.com/
